In [1]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.1.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: c:\users\svksh\deploy\lib\site-packages
Requires: scipy, numpy, joblib, threadpoolctl
Required-by: 


In [2]:
import pickle
import pandas as pd
import statistics

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

c:\Users\svksh\deploy\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\svksh\deploy\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
year = 2022
month = 4

In [14]:
input_file = './data/yellow_tripdata_{}-0{}.parquet'.format(year, month)
print(input_file)

./data/yellow_tripdata_2022-04.parquet


In [20]:
df = read_data(input_file)

In [21]:
# df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [22]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-04-01 00:21:13,2022-04-01 00:58:33,1.0,10.30,1.0,N,163,62,1,33.5,3.0,0.5,7.45,0.0,0.3,44.75,2.5,0.0,37.333333
1,1,2022-04-01 00:07:47,2022-04-01 00:19:12,0.0,2.00,1.0,N,142,141,1,10.0,3.0,0.5,4.10,0.0,0.3,17.90,2.5,0.0,11.416667
2,1,2022-04-01 00:14:52,2022-04-01 00:23:43,1.0,1.00,1.0,N,79,148,1,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,0.0,8.850000
3,1,2022-04-01 00:30:02,2022-04-01 00:45:06,1.0,2.60,1.0,N,79,13,1,12.0,3.0,0.5,4.70,0.0,0.3,20.50,2.5,0.0,15.066667
4,2,2022-04-01 00:48:40,2022-04-01 01:03:34,1.0,2.79,1.0,N,238,116,4,-12.5,-0.5,-0.5,0.00,0.0,-0.3,-13.80,0.0,0.0,14.900000


In [23]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [24]:
print(statistics.mean(y_pred))

12.865128336784922


In [32]:
# Standard deviation of preds
res = statistics.pstdev(y_pred)
 
# Printing result
print("Standard deviation of the predicted duration for this dataset : " + str(res))

Standard deviation of the predicted duration for this dataset : 5.2814035765533385


In [34]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['pred'] = y_pred

In [35]:
output_file = 'data/output.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)